In [1]:
import pandas as pd
import numpy as np
import mne
import re
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchsummary import summary
import torch.autograd as autograd

In [2]:
DATA_PATH = '../data/event_data.csv'
MFF_DIR = '../data/'
STIM_CHANNEL_NAMES = ['101' + str(i) for i in range(10)]
EEG_CHANNEL_NAMES = ['E'+ str(i) for i in range(1,33)]
FILE_ORDER = [
    'ABI-Ashutosh-25-Sept-2019_20190925_102455.mff',
    'ABI-Kartikeya-22-10-19_20191022_033359.mff',
    'ABI-Kartikeya-22-10-19_20191022_040406.mff',
    'ABI-Pranali-23-09-19_20190923_103745.mff',
    'ABI-Shreyas-21-09-19_20190921_033411.mff',
    'ABI2-Ashutosh-25-Sept-2019_20190925_104849.mff',
    'ABI2-Pranali-23-Sept-19_20190923_110343.mff',
    'ABI2-Shreyas-Sept-21_20190921_035826.mff',
    'ABI2_Nishit_16Oct-2019_20191016_111224.mff',
    'ABI_Nishit_16Oct-2019_20191016_104307.mff',  
]
EVENT_LENGTHS = [160] + [200]*9
NUMPY_X_FNAME, NUMPY_Y_FNAME = MFF_DIR + 'X_small.npy',MFF_DIR + 'y_small.npy'

In [3]:
######### NO NEED TO RUN ###################

In [4]:
class EEGDataset(Dataset):
    def __init__(self):
        self.df = pd.read_csv(DATA_PATH)
        self.current_file = None
        self.current_raw = None
        self.current_events = None
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        fname = self.df.iloc[idx]['fname']
        if self.current_file != fname or self.current_file is None:
            self.current_file = fname
            self.current_raw = mne.io.read_raw_egi(MFF_DIR + fname,verbose=False,preload=True).pick_channels(
                STIM_CHANNEL_NAMES+EEG_CHANNEL_NAMES)
            self.current_events = mne.find_events(self.current_raw,verbose=False)
        s_time = self.df.iloc[idx]['s_time']/1000.0
        if idx < 160:
            fake_idx = idx
        else:
            fake_idx = (idx - 160)%200 
        epoch = mne.Epochs(
            self.current_raw.copy().pick_types(eeg=True),
            self.current_events[fake_idx].reshape(1,-1),tmin=s_time,tmax=s_time+4.0-1.0/self.current_raw.info['sfreq'],
            baseline=None,verbose=False,preload=True)
        try:
            X = epoch.load_data().filter(l_freq = 0, h_freq = 30).resample(100).get_data()
        except ValueError:
            X = np.array([]).reshape(0,32,400)
        y = self.df.iloc[idx]['label']
        
        return X, y

In [5]:
eeg_dataset = EEGDataset()

In [6]:
X = []
y = []
i = 0
for xi,yi in eeg_dataset:
    if xi.shape == (1,32,400):
        print(i)
        X.append(xi)
        y.append(yi)
    i += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450


In [8]:
X_np = np.concatenate(X, axis=0)
y_np = np.array(y)
print(X_np.shape,y_np.shape)

(967, 32, 400) (967,)


In [9]:
np.save(NUMPY_X_FNAME,X_np)
np.save(NUMPY_Y_FNAME,y_np)

In [10]:
#####################################

In [11]:
X_np = np.load(NUMPY_X_FNAME)
y_np = np.load(NUMPY_Y_FNAME)

In [31]:
X_torch = torch.Tensor(X_np)
y_torch = torch.Tensor(y_np)
dset = TensorDataset(X_torch[:10],y_torch[:10]) # create your datset
dloader = DataLoader(dset,shuffle=True,batch_size=1) # create your dataloader

In [32]:
print(X_torch.shape,y_torch.shape)

torch.Size([967, 32, 400]) torch.Size([967])


In [33]:
# build model
class EEGEncoder(nn.Module):
    def __init__(self, n_i):
        super(EEGEncoder,self).__init__();
        self.n_i = n_i
        
        self.lstm1 = nn.LSTM(n_i, n_i, 1, bidirectional=False)
        self.linear2 = nn.Linear(n_i, n_i)
        self.linear3 = nn.Linear(n_i,n_i)
        self.linear4 = nn.Linear(n_i,10)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, x):
        x = x.reshape(400,-1,32)
        x, (hn,cn) = self.lstm1(x)
        enc = self.linear2(hn)
        x = F.relu(enc)
        x = self.linear3(x)
        x = self.linear4(x)
        x = self.softmax(x)
        
        return x

In [34]:
# Testing dimensions
eeg_classifier = EEGEncoder(32)
inp,l = torch.randn(400,2,32),torch.randn(2)
out = eeg_classifier(inp)
print(out.shape, inp.shape)

torch.Size([1, 2, 10]) torch.Size([400, 2, 32])


In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = opt.Adam(eeg_classifier.parameters(),lr=0.001)

In [42]:
losses = []
for epoch in range(5):
    running_loss = 0.0
    
    for i, data in enumerate(dloader,0):
        inp,lab = data
        optimizer.zero_grad()
        
        out = eeg_classifier(inp)
        out = out.reshape(-1,10)
                        
        loss = criterion(out.float(),lab.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1 == 0:
            print('[{}/{}]: loss = {}'.format(epoch+1,i+1,running_loss))
            losses.append(running_loss)
            running_loss = 0.0
            
print('Finished training')

[1/1]: loss = 2.4611501693725586
[1/2]: loss = 2.4611501693725586
[1/3]: loss = 1.4611501693725586
[1/4]: loss = 2.4611501693725586
[1/5]: loss = 1.4611501693725586
[1/6]: loss = 2.4611501693725586
[1/7]: loss = 2.4611501693725586
[1/8]: loss = 2.4611501693725586
[1/9]: loss = 2.4611501693725586
[1/10]: loss = 2.4611501693725586
[2/1]: loss = 2.4611501693725586
[2/2]: loss = 2.4611501693725586
[2/3]: loss = 1.4611501693725586
[2/4]: loss = 2.4611501693725586
[2/5]: loss = 2.4611501693725586
[2/6]: loss = 2.4611501693725586
[2/7]: loss = 2.4611501693725586
[2/8]: loss = 2.4611501693725586
[2/9]: loss = 2.4611501693725586
[2/10]: loss = 1.4611501693725586
[3/1]: loss = 2.4611501693725586
[3/2]: loss = 2.4611501693725586
[3/3]: loss = 1.4611501693725586
[3/4]: loss = 1.4611501693725586
[3/5]: loss = 2.4611501693725586
[3/6]: loss = 2.4611501693725586
[3/7]: loss = 2.4611501693725586
[3/8]: loss = 2.4611501693725586
[3/9]: loss = 2.4611501693725586
[3/10]: loss = 2.4611501693725586
[4/1]: 

In [22]:
torch.save(eeg_classifier.state_dict(), '../ckpt/save1.pth')

In [43]:
for i, data in enumerate(dloader,0):
        inp,lab = data
        out = eeg_classifier(inp);
        print(out.argmax(dim=2),lab)

tensor([[6]]) tensor([5.])
tensor([[6]]) tensor([6.])
tensor([[6]]) tensor([0.])
tensor([[6]]) tensor([9.])
tensor([[6]]) tensor([7.])
tensor([[6]]) tensor([6.])
tensor([[6]]) tensor([9.])
tensor([[6]]) tensor([2.])
tensor([[6]]) tensor([2.])
tensor([[6]]) tensor([0.])
